In [ ]:
import pandas as pd

In [ ]:
%reload_ext rpy2.ipython

/usr/local/lib/python3.6/dist-packages/rpy2/robjects/pandas2ri.py:14: FutureWarning: pandas.core.index is deprecated and will be removed in a future version.  The public classes are available in the top-level namespace.
  from pandas.core.index import Index as PandasIndex
/usr/local/lib/python3.6/dist-packages/rpy2/robjects/pandas2ri.py:34: UserWarning: pandas >= 1.0 is not supported.
  warnings.warn('pandas >= 1.0 is not supported.')


In [ ]:
%%R
install.packages("googledrive")
install.packages("tidyverse")
install.packages("broom")
install.packages("survminer")
install.packages("survival")
install.packages("cowplot")

In [ ]:
%%R
library(googledrive)
library(tidyverse)
library(tidyverse, warn.conflicts = FALSE)
library(lubridate, warn.conflicts = FALSE)
library(broom)
library(survminer)
library(survival)
library(cowplot)

In [ ]:
%%R
myPath <- "/content/drive/Shareddrives/Projeto Malaria - Artefatos/Datasets/SIVEP-MALARIA/01_RAW/CSV/UFs_2007_2019_Filter_UF-INFEC_Positive/Without Missing and Inconsistencies/"
files <- dir(path=myPath, pattern = "df_UF")
files

map(paste(myPath, "/", files, sep=""), read_lines, n_max = 2)

sivep <- map_dfr(paste(myPath, "/", files, sep=""), data.table::fread)
glimpse(sivep)
write_rds(sivep, "sivep_completo.rds")


In [ ]:
%%R
options(scipen = 6, na.action = 'na.exclude')

df <- read_rds("sivep_completo.rds")
df <- as_tibble(df)

# Renomeando os fatores -----
# Ocupação
df$COD_OCUP <- factor(df$COD_OCUP,
                      levels = c(1:11),
                      labels = c("Agricultura",
                                 "Pecuária",
                                 "Doméstica",
                                 "Turismo",
                                 "Garimpagem",
                                 "Exploração Vegetal",
                                 "Caça/Pesca",
                                 "Construção de estradas/barragens",
                                 "Mineração",
                                 "Viajante",
                                 "Outros"))

# Escolaridade
df$NIV_ESCO <- factor(df$NIV_ESCO,
                      levels = c(0:8),
                      labels = c("Analfabeto",
                                 "1ª a 4ª série incompleta do EF",
                                 "4ª série completa do EF",
                                 "5ª a 8ª série incompleta do EF",
                                 "Ensino fundamental completo",
                                 "Ensino médio incompleto",
                                 "Ensino médio completo",
                                 "Educação superior incompleto",
                                 "Educação superior completa"))

# Estado gestacional
df$GESTANTE. <- factor(df$GESTANTE.,
                       levels = c(1:5),
                       labels = c("1º Trimestre",
                                  "2º Trimestre",
                                  "3º Trimestre",
                                  "Idade gestacional ignorada",
                                  "Não"))

# Raça
df$RACA <- factor(df$RACA,
                  levels = c(1:5),
                  labels = c("Branca", "Preta", "Amarela", "Parda", "Indigena"))

# Quantidade de cruzes
df$QTD_CRUZ <- factor(df$QTD_CRUZ,
                      levels = c(1:6),
                      labels = c("< +/2",
                                 "+/2",
                                 "+",
                                 "++",
                                 "+++",
                                 "++++"))

# Resultado do exame
df$RES_EXAM <- factor(df$RES_EXAM,
                      levels = c(2:11),
                      labels = c("Falciparum",
                                 "F+Fg",
                                 "Vivax",
                                 "F+V",
                                 "V+Fg",
                                 "Fg",
                                 "Malariae",
                                 "F+M",
                                 "Ovale",
                                 "Não Falciparum"))

# Tipo de detecção
df$TIPO_LAM <- factor(df$TIPO_LAM,
                      levels = c(1:3),
                      labels = c("Detecção Passiva", "Detecção Ativa", "LVC"))

# Esquema
df$ESQUEMA <- factor(df$ESQUEMA,
                     levels = c(1:12, 83, 85:89, 99),
                     labels = c("Infecções pelo P. vivax, ou P. ovale com cloroquina em 3 dias e primaquina em 7 dias (esquema curto)",
                                "Infecções pelo P. vivax, ou P. ovale com cloroquina em 3 dias e primaquina em 14 dias (esquema longo)",
                                "Infecções pelo P. malariae para todas as idades e por P. vivax ou P. ovale em gestantes e crianças com menos de 6 meses, com cloroquina em 3 dias",
                                "Prevenção das recaídas frequentes por P. vivax ou P. ovale com cloroquina semanal em 12 semanas",
                                "Infecções por P. falciparum com a combinação fixa de artemeter+lumefantrina em 3 dias",
                                "Infecções por P. falciparum com a combinação fixa de artesunato+mefloquina em 3 dias",
                                "Infecções por P. falciparum com quinina em 3 dias, doxiciclina em 5 dias e primaquina no 6º dia)",
                                "Infecções mistas por P. falciparum e P. vivax ou P. ovale com Artemeter + Lumefantrina ou Artesunato + Mefloquina em 3 dias e Primaquina em 7 dias",
                                "Infecções não complicadas por P. falciparum no 1º trimestre da gestação e crianças com menos de 6 meses, com quinina em 3 dias e clindamicina em 5",
                                "Malária grave e complicada pelo P. falciparum em todas as faixas etárias",
                                "Infecções por P. falciparum com a combinação fixa de artemeter+lumefantrina em 3 dias e primaquina em dose única",
                                "Infecções por P. falciparum com a combinação fixa de artesunato+mefloquina em 3 dias e primaquina em dose única",
                                "Infecções mistas por Pv + Pf com Mefloquina em dose única e primaquina em 7 dias",
                                "Infecções por Pv em crianças apresentando vômitos, com cápsulas retais de artesunato em 4 dias e Primaquina em 7 dias",
                                "Infecções por Pf com Mefloquina em dose única e primaquina no segundo dia",
                                "Infecções por Pf com Quinina em 7 dias",
                                "Infecções por Pf de crianças com cápsulas retais de artesunato em 4 dias e dose única de Mefloquina no 3º dia e Primaquina no 5º ida",
                                "Infecções mistas por Pv + Pf com Quinina em 3 dias, doxiciclina em 5 dias e Primaquina em 7 dias",
                                "Outro esquema utilizado (por médico)"))

# Sexo
df$SEXO <- factor(df$SEXO, levels = c("M", "F"), labels = c("Masculino", "Feminino"))

# Id_lvc
df$ID_LVC <- factor(df$ID_LVC, levels = c(1:2), labels = c("LVC", "Não LVC"))

# Exame
df$EXAME <- factor(df$EXAME, levels = c(1:2), labels = c("Gota espessa/esfregaço", "Teste rápido"))

# Falciparum
df$FALCIPARUM <- factor(df$FALCIPARUM, levels = c(1:2), labels = c("Sim", "Não"))

# Vivax
df$VIVAX <- factor(df$VIVAX, levels = c(1:2), labels = c("Sim", "Não"))

# Hemoparasitas
df$HEMOPARASI <- factor(df$HEMOPARASI, levels = c(1:4, 9), labels = c("Negativo", "Trypanosoma sp.", "Microfilária", "Trypanosoma sp.+ Microfilária", "Não pesquisados"))

# Idade ou grupo de idade
df$ID_PACIE <- ifelse(df$ID_DIMEA != "A", 0, df$ID_PACIE)
df$age_groups <- cut(df$ID_PACIE, breaks = c(0, 5, 40, 60, Inf), right = FALSE)

dfBck <- df

dfBck

# A tibble: 3,575,089 x 53
   `Unnamed: 0` COD_NOTI COD_OCUP COD_UNIN DT_EXAME DT_NASCI DT_NOTIF DT_SINTO
          <int>    <int> <fct>       <int> <chr>    <chr>    <chr>    <chr>   
 1            0      248 Outros        148 2007-07… 1967-02… 2007-07… 2007-07…
 2            1      106 Viajante     4928 2007-05… 1960-11… 2007-05… 2007-05…
 3            2      266 Viajante     4928 2007-10… 1956-05… 2007-10… 2007-10…
 4            3      123 Viajante     1430 2007-10… 1949-10… 2007-10… 2007-10…
 5            4      130 Explora…     1430 2007-10… 1972-02… 2007-10… 2007-10…
 6            5       10 Agricul…     4013 2007-03… 1982-10… 2007-03… 2007-03…
 7            6       13 Agricul…     4013 2007-03… 1988-07… 2007-03… 2007-02…
 8            7       39 Turismo   3028925 2007-08… 1972-11… 2007-08… 2007-08…
 9            8       41 Agricul…  3028925 2007-08… 1972-11… 2007-08… 2007-08…
10           54     4110 Agricul…       12 2007-12… 2007-07… 2007-12… 2007-12…
# … with 3,575,079 more r

In [ ]:
%%R

df <- dfBck

# Taxa de incidência anual

#read_lines("../banco-mun/Dataset_municip_TxIncidencia.csv")
ipa <- read_csv("Dataset_municip_TxIncidencia.csv")
ipa <- ipa %>% mutate(grp_risc = cut(
  txinc, breaks = c(0, 10, 50, Inf), labels = c("Baixo", "Medio", "Alto")))

df <- left_join(df, ipa, by = c("MUN_INFE", "year_notif"))

# Apenas pacientes com data dos sintomas e data do início do tratamento
# Apenas pacientes com data dos sintomas ANTERIOR a data do tratamento
# Apenas notificações primárias
df <- df %>% filter(
  !is.na(date_sinto),
  !is.na(date_trata),
  date_sinto <= date_trata,
  ID_LVC == "Não LVC")

# Censura a esquerda, pacientes que levaram mais de 48h para iniciar o atendimento
df <- df %>% mutate(
  tempo = as.duration(date_sinto %--% date_trata) / ddays(1), # Variável tempo
  year = factor(year(date_trata)),
  importado = factor(
    ifelse(MUN_INFE == MUN_RESI, TRUE, FALSE),
    c(TRUE, FALSE),
    c("Autóctone", "Importado")),
  trat = case_when(
    tempo <= 2 & importado == "Autóctone" ~ 1L,
    tempo <= 4 & importado == "Importado" ~ 1L,
    TRUE ~ 0L)) # Variável tratamento rápido (48h ou 96h)

# Variáveis de estudo
df <- df %>% select(
  tempo, trat, age_groups, year, grp_risc, importado, COD_OCUP, ESQUEMA, EXAME, FALCIPARUM,
  GESTANTE., HEMOPARASI, ID_LVC, ID_PACIE, NIV_ESCO, QTD_CRUZ, RACA, RES_EXAM,
  SEXO, TIPO_LAM, VIVAX)

df_auto <- df %>% filter(importado == "Autóctone")
df_imp <- df %>% filter(importado == "Importado")


# html(describe(df), size = 80, scroll = TRUE)

# Informal testing
# df %>% filter(importado == "Autóctone") %>% sample_n(10)

R[write to console]: 
── Column specification ────────────────────────────────────────────────────────
cols(
  MUN_INFE = col_double(),
  year_notif = col_double(),
  txinc = col_double()
)




In [ ]:
%%R
df_auto

# A tibble: 2,444,001 x 21
   tempo  trat age_groups year  grp_risc importado COD_OCUP ESQUEMA EXAME
   <dbl> <int> <fct>      <fct> <fct>    <fct>     <fct>    <fct>   <fct>
 1     1     1 [5,40)     2007  Alto     Autóctone Outros   Infecç… <NA> 
 2     2     1 [60,Inf)   2007  Alto     Autóctone Agricul… Infecç… <NA> 
 3     7     0 [40,60)    2007  Alto     Autóctone Outros   Infecç… <NA> 
 4     1     1 [5,40)     2007  Alto     Autóctone Outros   Infecç… <NA> 
 5     2     1 [5,40)     2007  Alto     Autóctone Outros   Infecç… <NA> 
 6     3     0 [5,40)     2007  Alto     Autóctone Outros   Infecç… <NA> 
 7     0     1 [5,40)     2007  Alto     Autóctone Outros   Outro … <NA> 
 8     2     1 [40,60)    2007  Alto     Autóctone Outros   Infecç… <NA> 
 9     0     1 [0,5)      2007  Baixo    Autóctone <NA>     Outro … <NA> 
10     3     0 [5,40)     2007  Alto     Autóctone Agricul… Infecç… <NA> 
# … with 2,443,991 more rows, and 12 more variables: FALCIPARUM <fct>,
#   GESTANTE. 

In [ ]:
%%R

# Kaplan-Meier ----
km_fit_a <- survfit(Surv(tempo, trat) ~ 1, data = df_auto)
km_fit_i <- survfit(Surv(tempo, trat) ~ 1, data = df_imp)

survdiff()

lapply(list(km_fit_a, km_fit_i), summary)

R[write to console]: Error in inherits(formula, "formula") : 
  argument "formula" is missing, with no default
Calls: <Anonymous> -> <Anonymous> -> withVisible -> survdiff




Error in inherits(formula, "formula") : 
  argument "formula" is missing, with no default
Calls: <Anonymous> -> <Anonymous> -> withVisible -> survdiff


In [ ]:
# Plot
g <- ggsurvplot(km_fit_a,
                xlim = c(0, 5),
                break.time.by = 1,
                linetype = "strata",
                title = "Autóctones",
                xlab = "Dias", ylab = "Sobrev.",
                legend = 'none',
                palette = "jco",
                ggtheme = theme_light())

h <- ggsurvplot(km_fit_i,
                xlim = c(0, 5),
                break.time.by = 1,
                linetype = "strata",
                title = "Importados",
                xlab = "Dias", ylab = NULL,
                legend = 'none',
                palette = "jco",
                ggtheme = theme_light())

p1 <- g$plot +
  geom_hline(yintercept = 0.3) +
  geom_text(aes(x = 0.1, y = 0.3, label = "Meta"), nudge_y = 0.02) +
  geom_vline(xintercept = 2) +
  geom_text(aes(x = 2, y = 0.05, label = "Prazo", angle = 90), nudge_x = -0.15) +
  theme(legend.position = 'none')
p2 <- h$plot +
  geom_hline(yintercept = 0.3) +
  geom_text(aes(x = 0.1, y = 0.3, label = "Meta"), nudge_y = 0.02) +
  geom_vline(xintercept = 4) +
  geom_text(aes(x = 4, y = 0.05, label = "Prazo", angle = 90), nudge_x = -0.15) +
  theme(legend.position = 'none')

title <- ggdraw() +
  draw_label("Kaplan-Meier sem covariáveis",
             fontface = 'bold',
             x = 0,
             hjust = 0) +
  # add margin on the left of the drawing canvas,
  # so title is aligned with left edge of first plot
  theme(plot.margin = margin(0, 0, 0, 7))

plot_row <- plot_grid(p1, p2)

grid <- plot_grid(title, plot_row, ncol = 1, rel_heights = c(0.1, 1))

ggsave("km-geral.png", print(grid), width = 8, height = 6)

file.show("km-geral.png")

# Kaplan-Meier estratificado ----
# Grupo de risco
km_fit_a <- survfit(data = df_auto, Surv(tempo, trat) ~ grp_risc)
km_fit_i <- survfit(data = df_imp, Surv(tempo, trat) ~ grp_risc)

lapply(list(km_fit_a, km_fit_i), summary)

# Plot
g <- ggsurvplot(km_fit_a,
                xlim = c(0, 5),
                break.time.by = 1,
                linetype = "strata",
                title = "Autóctones",
                xlab = "Dias", ylab = "Sobrev.",
                legend = 'right',
                palette = "jco",
                ggtheme = theme_light())

h <- ggsurvplot(km_fit_i,
                xlim = c(0, 5),
                break.time.by = 1,
                linetype = "strata",
                title = "Importados",
                xlab = "Dias", ylab = NULL,
                palette = "jco",
                ggtheme = theme_light())

p1 <- g$plot +
  geom_hline(yintercept = 0.3) +
  geom_text(aes(x = 0.1, y = 0.3, label = "Meta"), nudge_y = 0.02) +
  geom_vline(xintercept = 2) +
  geom_text(aes(x = 2, y = 0.05, label = "Prazo", angle = 90), nudge_x = -0.15) +
  theme(legend.position = 'none')
p2 <- h$plot +
  geom_hline(yintercept = 0.3) +
  geom_text(aes(x = 0.1, y = 0.3, label = "Meta"), nudge_y = 0.02) +
  geom_vline(xintercept = 4) +
  geom_text(aes(x = 4, y = 0.05, label = "Prazo", angle = 90), nudge_x = -0.15) +
  theme(legend.position = 'none')

title <- ggdraw() +
  draw_label(label = "Kaplan-Meier estratificado por grupo de risco do município",
             fontface = 'bold',
             x = 0,
             hjust = 0) +
  # add margin on the left of the drawing canvas,
  # so title is aligned with left edge of first plot
  theme(plot.margin = margin(0, 0, 0, 7))

legend <- get_legend(g$plot)

plot_row <- plot_grid(p1, p2, legend, nrow = 1, rel_widths = c(1, 1, 1/2))

grid <- plot_grid(title, plot_row, ncol = 1, rel_heights = c(0.1, 1))

ggsave("km-grisc.png", print(grid), width = 8, height = 6)

file.show("km-grisc.png")

log_rank_a <- survdiff(data = df_auto, Surv(tempo, trat) ~ grp_risc)
log_rank_i <- survdiff(data = df_imp, Surv(tempo, trat) ~ grp_risc)

knitr::kable(glance(log_rank_a))
knitr::kable(glance(log_rank_i))

# Ano
df$year2 <- factor(df$year, c(2007, 2013, 2017, 2019))
df_auto$year2 <- factor(df_auto$year, c(2007, 2013, 2017, 2019))
df_imp$year2 <- factor(df_imp$year, c(2007, 2013, 2017, 2019))

km_fit_a <- survfit(data = df_auto, Surv(tempo, trat) ~ year2)
km_fit_i <- survfit(data = df_imp, Surv(tempo, trat) ~ year2)

lapply(list(km_fit_a, km_fit_i), summary)

g <- ggsurvplot(km_fit_a,
                xlim = c(0, 5),
                break.time.by = 1,
                linetype = "strata",
                title = "Autóctones",
                xlab = "Dias", ylab = "Sobrev.",
                legend = 'right',
                palette = "jco",
                ggtheme = theme_light())

h <- ggsurvplot(km_fit_i,
                xlim = c(0, 5),
                break.time.by = 1,
                linetype = "strata",
                title = "Importados",
                xlab = "Dias", ylab = NULL,
                palette = "jco",
                ggtheme = theme_light())

p1 <- g$plot +
  geom_hline(yintercept = 0.3) +
  geom_text(aes(x = 0.1, y = 0.3, label = "Meta"), nudge_y = 0.02) +
  geom_vline(xintercept = 2) +
  geom_text(aes(x = 2, y = 0.05, label = "Prazo", angle = 90), nudge_x = -0.15) +
  theme(legend.position = 'none')
p2 <- h$plot +
  geom_hline(yintercept = 0.3) +
  geom_text(aes(x = 0.1, y = 0.3, label = "Meta"), nudge_y = 0.02) +
  geom_vline(xintercept = 4) +
  geom_text(aes(x = 4, y = 0.05, label = "Prazo", angle = 90), nudge_x = -0.15) +
  theme(legend.position = 'none')

title <- ggdraw() +
  draw_label(label = "Kaplan-Meier estratificado por anos selecionados",
             fontface = 'bold', x = 0, hjust = 0) +
  # add margin on the left of the drawing canvas,
  # so title is aligned with left edge of first plot
  theme(plot.margin = margin(0, 0, 0, 7))

legend <- get_legend(g$plot)

plot_row <- plot_grid(p1, p2, legend, nrow = 1, rel_widths = c(1, 1, 1/2))

grid <- plot_grid(title, plot_row, ncol = 1, rel_heights = c(0.1, 1))

ggsave("km-ano.png", print(grid), width = 8, height = 6)

file.show("km-ano.png")

log_rank_a <- survdiff(data = df_auto, Surv(tempo, trat) ~ year2)
log_rank_i <- survdiff(data = df_imp, Surv(tempo, trat) ~ year2)

knitr::kable(glance(log_rank_a))
knitr::kable(glance(log_rank_i))

# Sexo
km_fit_a <- survfit(data = df_auto, Surv(tempo, trat) ~ SEXO)
km_fit_i <- survfit(data = df_imp, Surv(tempo, trat) ~ SEXO)

lapply(list(km_fit_a, km_fit_i), summary)

# Plot
g <- ggsurvplot(km_fit_a,
                xlim = c(0, 5),
                break.time.by = 1,
                title = "Autóctones",
                linetype = "strata",
                xlab = "Dias", ylab = "Sobrev.",
                legend = 'right',
                palette = "jco",
                ggtheme = theme_light())

h <- ggsurvplot(km_fit_i,
                xlim = c(0, 5),
                break.time.by = 1,
                linetype = "strata",
                title = "Importados",
                xlab = "Dias", ylab = NULL,
                palette = "jco",
                ggtheme = theme_light())

p1 <- g$plot +
  geom_hline(yintercept = 0.3) +
  geom_text(aes(x = 0.1, y = 0.3, label = "Meta"), nudge_y = 0.02) +
  geom_vline(xintercept = 2) +
  geom_text(aes(x = 2, y = 0.05, label = "Prazo", angle = 90), nudge_x = -0.15) +
  theme(legend.position = 'none')
p2 <- h$plot +
  geom_hline(yintercept = 0.3) +
  geom_text(aes(x = 0.1, y = 0.3, label = "Meta"), nudge_y = 0.02) +
  geom_vline(xintercept = 4) +
  geom_text(aes(x = 4, y = 0.05, label = "Prazo", angle = 90), nudge_x = -0.15) +
  theme(legend.position = 'none')

title <- ggdraw() +
  draw_label(label = "Kaplan-Meier estratificado por sexo",
             fontface = 'bold', x = 0, hjust = 0) +
  # add margin on the left of the drawing canvas,
  # so title is aligned with left edge of first plot
  theme(plot.margin = margin(0, 0, 0, 7))

legend <- get_legend(g$plot)

plot_row <- plot_grid(p1, p2, legend, nrow = 1, rel_widths = c(1, 1, 1/2))

grid <- plot_grid(title, plot_row, ncol = 1, rel_heights = c(0.1, 1))

ggsave("km-sexo.png", print(grid), width = 8, height = 6)

file.show("km-sexo.png")

log_rank_a <- survdiff(data = df_auto, Surv(tempo, trat) ~ SEXO)
log_rank_i <- survdiff(data = df_imp, Surv(tempo, trat) ~ SEXO)

knitr::kable(glance(log_rank_a))
knitr::kable(glance(log_rank_i))

# Ocupação
df_auto$ocup2 <- fct_lump_n(df_auto$COD_OCUP, n = 7, other_level = "Outros")
df_imp$ocup2 <- fct_lump_n(df_imp$COD_OCUP, n = 7, other_level = "Outros")

km_fit_a <- survfit(data = df_auto, Surv(tempo, trat) ~ ocup2)
km_fit_i <- survfit(data = df_imp, Surv(tempo, trat) ~ ocup2)

lapply(list(km_fit_a, km_fit_i), summary)

# Plot
g <- ggsurvplot(km_fit_a,
                xlim = c(0, 5),
                break.time.by = 1,
                linetype = "strata",
                title = "Autóctones",
                xlab = "Dias", ylab = "Sobrev.",
                legend = 'right',
                palette = "jco",
                ggtheme = theme_light())

h <- ggsurvplot(km_fit_i,
                xlim = c(0, 5),
                break.time.by = 1,
                linetype = "strata",
                title = "Importados",
                xlab = "Dias", ylab = NULL,
                palette = "jco",
                ggtheme = theme_light())

p1 <- g$plot +
  geom_hline(yintercept = 0.3) +
  geom_text(aes(x = 0.1, y = 0.3, label = "Meta"), nudge_y = 0.02) +
  geom_vline(xintercept = 2) +
  geom_text(aes(x = 2, y = 0.05, label = "Prazo", angle = 90), nudge_x = -0.15) +
  theme(legend.position = 'none')

p2 <- h$plot +
  geom_hline(yintercept = 0.3) +
  geom_text(aes(x = 0.1, y = 0.3, label = "Meta"), nudge_y = 0.02) +
  geom_vline(xintercept = 4) +
  geom_text(aes(x = 4, y = 0.05, label = "Prazo", angle = 90), nudge_x = -0.15) +
  theme(legend.position = 'none')

title <- ggdraw() +
  draw_label(label = "Kaplan-Meier estratificado por ocupação",
             fontface = 'bold', x = 0, hjust = 0) +
  # add margin on the left of the drawing canvas,
  # so title is aligned with left edge of first plot
  theme(plot.margin = margin(0, 0, 0, 7))

legend <- get_legend(g$plot)

plot_row <- plot_grid(p1, p2, legend, nrow = 1, rel_widths = c(1, 1, 1/2))

grid <- plot_grid(title, plot_row, ncol = 1, rel_heights = c(0.1, 1))

ggsave("km-ocup.png", print(grid), width = 8, height = 6)

file.show("km-ocup.png")

log_rank_a <- survdiff(data = df_auto, Surv(tempo, trat) ~ ocup2)
log_rank_i <- survdiff(data = df_imp, Surv(tempo, trat) ~ ocup2)

knitr::kable(glance(log_rank_a))
knitr::kable(glance(log_rank_i))

# Nível de escolaridade
df_auto$niv_esco <- fct_collapse(
  df_auto$NIV_ESCO,
  "Fundamental incompleto" = c("Analfabeto",
                               "1ª a 4ª série incompleta do EF",
                               "4ª série completa do EF",
                               "5ª a 8ª série incompleta do EF"),
  "Médio incompleto"       = c("Ensino fundamental completo", "Ensino médio incompleto"),
  "Superior incompleto"    = c("Ensino médio completo", "Educação superior incompleto"),
  "Superior completo"      = c("Educação superior completa"))

df_imp$niv_esco <- fct_collapse(
  df_imp$NIV_ESCO,
  "Fundamental incompleto" = c("Analfabeto",
                               "1ª a 4ª série incompleta do EF",
                               "4ª série completa do EF",
                               "5ª a 8ª série incompleta do EF"),
  "Médio incompleto"       = c("Ensino fundamental completo", "Ensino médio incompleto"),
  "Superior incompleto"    = c("Ensino médio completo", "Educação superior incompleto"),
  "Superior completo"      = c("Educação superior completa"))


km_fit_a <- survfit(data = df_auto, Surv(tempo, trat) ~ niv_esco)
km_fit_i <- survfit(data = df_imp, Surv(tempo, trat) ~ niv_esco)

lapply(list(km_fit_a, km_fit_i), summary)

# Plot
g <- ggsurvplot(km_fit_a,
                xlim = c(0, 5),
                break.time.by = 1,
                linetype = "strata",
                title = "Autóctones",
                xlab = "Dias", ylab = "Sobrev.",
                legend = 'right',
                palette = "jco",
                ggtheme = theme_light())

h <- ggsurvplot(km_fit_i,
                xlim = c(0, 5),
                break.time.by = 1,
                linetype = "strata",
                title = "Importados",
                xlab = "Dias", ylab = NULL,
                palette = "jco",
                ggtheme = theme_light())

p1 <- g$plot +
  geom_hline(yintercept = 0.3) +
  geom_text(aes(x = 0.1, y = 0.3, label = "Meta"), nudge_y = 0.02) +
  geom_vline(xintercept = 2) +
  geom_text(aes(x = 2, y = 0.05, label = "Prazo", angle = 90), nudge_x = -0.15) +
  theme(legend.position = 'none')

p2 <- h$plot +
  geom_hline(yintercept = 0.3) +
  geom_text(aes(x = 0.1, y = 0.3, label = "Meta"), nudge_y = 0.02) +
  geom_vline(xintercept = 4) +
  geom_text(aes(x = 4, y = 0.05, label = "Prazo", angle = 90), nudge_x = -0.15) +
  theme(legend.position = 'none')

title <- ggdraw() +
  draw_label(label = "Kaplan-Meier estratificado por nível de escolaridade",
             fontface = 'bold', x = 0, hjust = 0) +
  # add margin on the left of the drawing canvas,
  # so title is aligned with left edge of first plot
  theme(plot.margin = margin(0, 0, 0, 7))

legend <- get_legend(g$plot)

plot_row <- plot_grid(p1, p2, legend, nrow = 1, rel_widths = c(1, 1, 1/2))

grid <- plot_grid(title, plot_row, ncol = 1, rel_heights = c(0.1, 1))

ggsave("km-esco.png", print(grid), width = 10, height = 6)

file.show("km-esco.png")

log_rank_a <- survdiff(data = df_auto, Surv(tempo, trat) ~ niv_esco)
log_rank_i <- survdiff(data = df_imp, Surv(tempo, trat) ~ niv_esco)

knitr::kable(glance(log_rank_a))
knitr::kable(glance(log_rank_i))

# Gestante
km_fit_a <- survfit(data = df_auto, Surv(tempo, trat) ~ GESTANTE.)
km_fit_i <- survfit(data = df_imp, Surv(tempo, trat) ~ GESTANTE.)

lapply(list(km_fit_a, km_fit_i), summary)

# Plot
g <- ggsurvplot(km_fit_a,
                xlim = c(0, 5),
                break.time.by = 1,
                linetype = "strata",
                title = "Autóctones",
                xlab = "Dias", ylab = "Sobrev.",
                legend = 'right',
                palette = "jco",
                ggtheme = theme_light())

h <- ggsurvplot(km_fit_i,
                xlim = c(0, 5),
                break.time.by = 1,
                linetype = "strata",
                title = "Importados",
                xlab = "Dias", ylab = NULL,
                palette = "jco",
                ggtheme = theme_light())

p1 <- g$plot +
  geom_hline(yintercept = 0.3) +
  geom_text(aes(x = 0.1, y = 0.3, label = "Meta"), nudge_y = 0.02) +
  geom_vline(xintercept = 2) +
  geom_text(aes(x = 2, y = 0.05, label = "Prazo", angle = 90), nudge_x = -0.15) +
  theme(legend.position = 'none')

p2 <- h$plot +
  geom_hline(yintercept = 0.3) +
  geom_text(aes(x = 0.1, y = 0.3, label = "Meta"), nudge_y = 0.02) +
  geom_vline(xintercept = 4) +
  geom_text(aes(x = 4, y = 0.05, label = "Prazo", angle = 90), nudge_x = -0.15) +
  theme(legend.position = 'none')

title <- ggdraw() +
  draw_label(label = "Kaplan-Meier estratificado por estado gestacional",
             fontface = 'bold', x = 0, hjust = 0) +
  # add margin on the left of the drawing canvas,
  # so title is aligned with left edge of first plot
  theme(plot.margin = margin(0, 0, 0, 7))

legend <- get_legend(g$plot)

plot_row <- plot_grid(p1, p2, legend, nrow = 1, rel_widths = c(1, 1, 3/4))

grid <- plot_grid(title, plot_row, ncol = 1, rel_heights = c(0.1, 1))

ggsave("km-gest.png", print(grid), width = 10, height = 6)

file.show("km-gest.png")

log_rank_a <- survdiff(data = df_auto, Surv(tempo, trat) ~ GESTANTE.)
log_rank_i <- survdiff(data = df_imp, Surv(tempo, trat) ~ GESTANTE.)

knitr::kable(glance(log_rank_a))
knitr::kable(glance(log_rank_i))

# Idade
km_fit_a <- survfit(data = df_auto, Surv(tempo, trat) ~ age_groups)
km_fit_i <- survfit(data = df_imp, Surv(tempo, trat) ~ age_groups)

lapply(list(km_fit_a, km_fit_i), summary)

# Plot
g <- ggsurvplot(km_fit_a,
                xlim = c(0, 5),
                break.time.by = 1,
                linetype = "strata",
                title = "Autóctones",
                xlab = "Dias", ylab = "Sobrev.",
                legend = 'right',
                palette = "jco",
                ggtheme = theme_light())

h <- ggsurvplot(km_fit_i,
                xlim = c(0, 5),
                break.time.by = 1,
                linetype = "strata",
                title = "Importados",
                xlab = "Dias", ylab = NULL,
                palette = "jco",
                ggtheme = theme_light())

p1 <- g$plot +
  geom_hline(yintercept = 0.3) +
  geom_text(aes(x = 0.1, y = 0.3, label = "Meta"), nudge_y = 0.02) +
  geom_vline(xintercept = 2) +
  geom_text(aes(x = 2, y = 0.05, label = "Prazo", angle = 90), nudge_x = -0.15) +
  theme(legend.position = 'none')

p2 <- h$plot +
  geom_hline(yintercept = 0.3) +
  geom_text(aes(x = 0.1, y = 0.3, label = "Meta"), nudge_y = 0.02) +
  geom_vline(xintercept = 4) +
  geom_text(aes(x = 4, y = 0.05, label = "Prazo", angle = 90), nudge_x = -0.15) +
  theme(legend.position = 'none')

title <- ggdraw() +
  draw_label(label = "Kaplan-Meier estratificado por grupo etário",
             fontface = 'bold', x = 0, hjust = 0) +
  # add margin on the left of the drawing canvas,
  # so title is aligned with left edge of first plot
  theme(plot.margin = margin(0, 0, 0, 7))

legend <- get_legend(g$plot)

plot_row <- plot_grid(p1, p2, legend, nrow = 1, rel_widths = c(1, 1, 3/4))

grid <- plot_grid(title, plot_row, ncol = 1, rel_heights = c(0.1, 1))

ggsave("km-age.png", print(grid), width = 10, height = 6)

file.show("km-age.png")

log_rank_a <- survdiff(data = df_auto, Surv(tempo, trat) ~ age_groups)
log_rank_i <- survdiff(data = df_imp, Surv(tempo, trat) ~ age_groups)

knitr::kable(glance(log_rank_a))
knitr::kable(glance(log_rank_i))

# Raça/Cor
km_fit_a <- survfit(data = df_auto, Surv(tempo, trat) ~ RACA)
km_fit_i <- survfit(data = df_imp, Surv(tempo, trat) ~ RACA)

lapply(list(km_fit_a, km_fit_i), summary)

# Plot
g <- ggsurvplot(km_fit_a,
                xlim = c(0, 5),
                break.time.by = 1,
                linetype = "strata",
                title = "Autóctones",
                xlab = "Dias", ylab = "Sobrev.",
                legend = 'right',
                palette = "jco",
                ggtheme = theme_light())

h <- ggsurvplot(km_fit_i,
                xlim = c(0, 5),
                break.time.by = 1,
                linetype = "strata",
                title = "Importados",
                xlab = "Dias", ylab = NULL,
                palette = "jco",
                ggtheme = theme_light())

p1 <- g$plot +
  geom_hline(yintercept = 0.3) +
  geom_text(aes(x = 0.1, y = 0.3, label = "Meta"), nudge_y = 0.02) +
  geom_vline(xintercept = 2) +
  geom_text(aes(x = 2, y = 0.05, label = "Prazo", angle = 90), nudge_x = -0.15) +
  theme(legend.position = 'none')

p2 <- h$plot +
  geom_hline(yintercept = 0.3) +
  geom_text(aes(x = 0.1, y = 0.3, label = "Meta"), nudge_y = 0.02) +
  geom_vline(xintercept = 4) +
  geom_text(aes(x = 4, y = 0.05, label = "Prazo", angle = 90), nudge_x = -0.15) +
  theme(legend.position = 'none')

title <- ggdraw() +
  draw_label(label = "Kaplan-Meier estratificado por raça/cor",
             fontface = 'bold', x = 0, hjust = 0) +
  # add margin on the left of the drawing canvas,
  # so title is aligned with left edge of first plot
  theme(plot.margin = margin(0, 0, 0, 7))

legend <- get_legend(g$plot)

plot_row <- plot_grid(p1, p2, legend, nrow = 1, rel_widths = c(1, 1, 3/4))

grid <- plot_grid(title, plot_row, ncol = 1, rel_heights = c(0.1, 1))

ggsave("km-raca.png", print(grid), width = 10, height = 6)

file.show("km-raca.png")

log_rank_a <- survdiff(data = df_auto, Surv(tempo, trat) ~ RACA)
log_rank_i <- survdiff(data = df_imp, Surv(tempo, trat) ~ RACA)

knitr::kable(glance(log_rank_a))
knitr::kable(glance(log_rank_i))

# Qtd cruz
km_fit_a <- survfit(data = df_auto, Surv(tempo, trat) ~ QTD_CRUZ)
km_fit_i <- survfit(data = df_imp, Surv(tempo, trat) ~ QTD_CRUZ)

lapply(list(km_fit_a, km_fit_i), summary)

# Plot
g <- ggsurvplot(km_fit_a,
                xlim = c(0, 5),
                break.time.by = 1,
                linetype = "strata",
                title = "Autóctones",
                xlab = "Dias", ylab = "Sobrev.",
                legend = 'right',
                palette = "jco",
                ggtheme = theme_light())

h <- ggsurvplot(km_fit_i,
                xlim = c(0, 5),
                break.time.by = 1,
                linetype = "strata",
                title = "Importados",
                xlab = "Dias", ylab = NULL,
                palette = "jco",
                ggtheme = theme_light())

p1 <- g$plot +
  geom_hline(yintercept = 0.3) +
  geom_text(aes(x = 0.1, y = 0.3, label = "Meta"), nudge_y = 0.02) +
  geom_vline(xintercept = 2) +
  geom_text(aes(x = 2, y = 0.05, label = "Prazo", angle = 90), nudge_x = -0.15) +
  theme(legend.position = 'none')

p2 <- h$plot +
  geom_hline(yintercept = 0.3) +
  geom_text(aes(x = 0.1, y = 0.3, label = "Meta"), nudge_y = 0.02) +
  geom_vline(xintercept = 4) +
  geom_text(aes(x = 4, y = 0.05, label = "Prazo", angle = 90), nudge_x = -0.15) +
  theme(legend.position = 'none')

title <- ggdraw() +
  draw_label(label = "Kaplan-Meier estratificado por quantidade de parasitas (cruzes)",
             fontface = 'bold', x = 0, hjust = 0) +
  # add margin on the left of the drawing canvas,
  # so title is aligned with left edge of first plot
  theme(plot.margin = margin(0, 0, 0, 7))

legend <- get_legend(g$plot)

plot_row <- plot_grid(p1, p2, legend, nrow = 1, rel_widths = c(1, 1, 3/4))

grid <- plot_grid(title, plot_row, ncol = 1, rel_heights = c(0.1, 1))

ggsave("km-cruz.png", print(grid), width = 10, height = 6)

file.show("km-cruz.png")

log_rank_a <- survdiff(data = df_auto, Surv(tempo, trat) ~ QTD_CRUZ)
log_rank_i <- survdiff(data = df_imp, Surv(tempo, trat) ~ QTD_CRUZ)

knitr::kable(glance(log_rank_a))
knitr::kable(glance(log_rank_i))

# Tipo de detecção
km_fit_a <- survfit(data = df_auto, Surv(tempo, trat) ~ TIPO_LAM)
km_fit_i <- survfit(data = df_imp, Surv(tempo, trat) ~ TIPO_LAM)

lapply(list(km_fit_a, km_fit_i), summary)

# Plot
g <- ggsurvplot(km_fit_a,
                xlim = c(0, 5),
                break.time.by = 1,
                linetype = "strata",
                title = "Autóctones",
                xlab = "Dias", ylab = "Sobrev.",
                legend = 'right',
                palette = "jco",
                ggtheme = theme_light())

h <- ggsurvplot(km_fit_i,
                xlim = c(0, 5),
                break.time.by = 1,
                linetype = "strata",
                title = "Importados",
                xlab = "Dias", ylab = NULL,
                palette = "jco",
                ggtheme = theme_light())

p1 <- g$plot +
  geom_hline(yintercept = 0.3) +
  geom_text(aes(x = 0.1, y = 0.3, label = "Meta"), nudge_y = 0.02) +
  geom_vline(xintercept = 2) +
  geom_text(aes(x = 2, y = 0.05, label = "Prazo", angle = 90), nudge_x = -0.15) +
  theme(legend.position = 'none')

p2 <- h$plot +
  geom_hline(yintercept = 0.3) +
  geom_text(aes(x = 0.1, y = 0.3, label = "Meta"), nudge_y = 0.02) +
  geom_vline(xintercept = 4) +
  geom_text(aes(x = 4, y = 0.05, label = "Prazo", angle = 90), nudge_x = -0.15) +
  theme(legend.position = 'none')

title <- ggdraw() +
  draw_label(label = "Kaplan-Meier estratificado por tipo de detecção",
             fontface = 'bold', x = 0, hjust = 0) +
  # add margin on the left of the drawing canvas,
  # so title is aligned with left edge of first plot
  theme(plot.margin = margin(0, 0, 0, 7))

legend <- get_legend(g$plot)

plot_row <- plot_grid(p1, p2, legend, nrow = 1, rel_widths = c(1, 1, 3/4))

grid <- plot_grid(title, plot_row, ncol = 1, rel_heights = c(0.1, 1))

ggsave("km-detec.png", print(grid), width = 10, height = 6)

file.show("km-detec.png")

log_rank_a <- survdiff(data = df_auto, Surv(tempo, trat) ~ TIPO_LAM)
log_rank_i <- survdiff(data = df_imp, Surv(tempo, trat) ~ TIPO_LAM)

knitr::kable(glance(log_rank_a))
knitr::kable(glance(log_rank_i))

# Parasita
df_auto$tipo_malaria <- fct_lump_n(df_auto$RES_EXAM, 3, other_level = "Outra")
df_imp$tipo_malaria <- fct_lump_n(df_imp$RES_EXAM, 3, other_level = "Outra")

km_fit_a <- survfit(data = df_auto, Surv(tempo, trat) ~ tipo_malaria)
km_fit_i <- survfit(data = df_imp, Surv(tempo, trat) ~ tipo_malaria)

lapply(list(km_fit_a, km_fit_i), summary)

# Plot
g <- ggsurvplot(km_fit_a,
                xlim = c(0, 5),
                break.time.by = 1,
                linetype = "strata",
                title = "Autóctones",
                xlab = "Dias", ylab = "Sobrev.",
                legend = 'right',
                palette = "jco",
                ggtheme = theme_light())

h <- ggsurvplot(km_fit_i,
                xlim = c(0, 5),
                break.time.by = 1,
                linetype = "strata",
                title = "Importados",
                xlab = "Dias", ylab = NULL,
                palette = "jco",
                ggtheme = theme_light())

p1 <- g$plot +
  geom_hline(yintercept = 0.3) +
  geom_text(aes(x = 0.1, y = 0.3, label = "Meta"), nudge_y = 0.02) +
  geom_vline(xintercept = 2) +
  geom_text(aes(x = 2, y = 0.05, label = "Prazo", angle = 90), nudge_x = -0.15) +
  theme(legend.position = 'none')

p2 <- h$plot +
  geom_hline(yintercept = 0.3) +
  geom_text(aes(x = 0.1, y = 0.3, label = "Meta"), nudge_y = 0.02) +
  geom_vline(xintercept = 4) +
  geom_text(aes(x = 4, y = 0.05, label = "Prazo", angle = 90), nudge_x = -0.15) +
  theme(legend.position = 'none')

title <- ggdraw() +
  draw_label(label = "Kaplan-Meier estratificado por tipo de parasita",
             fontface = 'bold', x = 0, hjust = 0) +
  # add margin on the left of the drawing canvas,
  # so title is aligned with left edge of first plot
  theme(plot.margin = margin(0, 0, 0, 7))

legend <- get_legend(g$plot)

plot_row <- plot_grid(p1, p2, legend, nrow = 1, rel_widths = c(1, 1, 3/4))

grid <- plot_grid(title, plot_row, ncol = 1, rel_heights = c(0.1, 1))

ggsave("km-parasi.png", print(grid), width = 10, height = 6)

file.show("km-parasi.png")

log_rank_a <- survdiff(data = df_auto, Surv(tempo, trat) ~ tipo_malaria)
log_rank_i <- survdiff(data = df_imp, Surv(tempo, trat) ~ tipo_malaria)

knitr::kable(glance(log_rank_a))
knitr::kable(glance(log_rank_i))